# RAG with Weaviate

In this example we are using my personal openAI Subscription using the  
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
!python --version

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aloiswirth/rag_examples/blob/master/weaviate_with_openai.ipynb)


## Prepare the environment

Check the environment we are in.

Install all necessary libraries

In [ ]:
get_ipython()

In [ ]:
RunningInCOLAB = 'google.colab' in str(get_ipython())
Running_locally = 'ZMQInteractiveShell' in (str(get_ipython()))

The next step is only needed for updating the Github repo.

In [ ]:
!pip --version
!pip install pip --upgrade
!pip --version

In [ ]:
# %pip install langchain
# %pip install requests
# %pip install weaviate-client
# %pip install sentence_transformers
%pip install openai



In [ ]:
import os
import requests
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from google.colab import userdata


Now we need to add from the google user secrets data the values for the "WEAVIATE_OPENAI_API_KEY" or we get it directly from the local environment variables.

In [ ]:
if Running_locally:
  OPENAI_API_KEY = os.environ["WEAVIATE_OPENAI_API_KEY"]
elif RunningInCOLAB:
  OPENAI_API_KEY = userdata.get("WEAVIATE_OPENAI_API_KEY")
else:
  print("please extend coding to cover this as well")


## Preparation
As a preparation step, you need to prepare a vector database as an external knowledge source that holds all additional information. This vector database is populated by following these steps:

- Collect and load your data
- Chunk your documents
- Embed and store chunks

In [ ]:
url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)


In [ ]:

loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

Next, chunk your documents — Because the Document, in its original state, is too long to fit into the LLM’s context window, you need to chunk it into smaller pieces. LangChain comes with many built-in text splitters for this purpose. For this simple example, you can use the CharacterTextSplitter with a chunk_size of about 500 and a chunk_overlap of 50 to preserve text continuity between the chunks.

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
chunks

Lastly, embed and store the chunks — To enable semantic search across the text chunks, you need to generate the vector embeddings for each chunk and then store them together with their embeddings. To generate the vector embeddings, you can use the OpenAI embedding model, and to store them, you can use the Weaviate vector database. By calling .from_documents() the vector database is automatically populated with the chunks.

In [ ]:
client = weaviate.Client(embedded_options=EmbeddedOptions())
vector_store = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = HuggingFaceBgeEmbeddings(),
    by_text = False
)

## Step 1: Retrieve
Once the vector database is populated, you can define it as the retriever component, which fetches the additional context based on the semantic similarity between the user query and the embedded chunks.

In [ ]:
retriever = vector_store.as_retriever()
retriever

## Step 2: Augment
Next, to augment the prompt with the additional context, you need to prepare a prompt template. The prompt can be easily customized from a prompt template, as shown below.

In [ ]:

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

## Step 3: Generate
Finally, you can build a chain for the RAG pipeline, chaining together the retriever, the prompt template and the LLM. Once the RAG chain is defined, you can invoke it.

In [ ]:


llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=OPENAI_API_KEY, temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

In [ ]:
# !git add -A
# !git commit -m "eliminating a security issue"
# !git push